# Live Trading Monitoring and Evaluation

This notebook demonstrates how to monitor and evaluate live trading performance using the BTB framework.

In [ ]:
import json
import os
import sys
import time
import warnings
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings("ignore")

# Add project root to path for imports
sys.path.append(os.path.abspath(".."))

from btb.backtest.metrics import calculate_metrics
from btb.exchange.factory import create_exchange
from btb.run.trader import Trader
from btb.strategies.factory import create_strategy
from btb.utils.config import load_config


## 1. Load Configuration

In [ ]:
# Load trading configuration
config = load_config("../config/trading_config.yaml")
print(f"Trading configuration loaded for {config['symbol']} with {config['timeframe']} timeframe")

## 2. Connect to Exchange

In [ ]:
# Initialize exchange from factory
exchange = create_exchange(
    exchange_type=config["exchange"],
    api_key=config["api_key"],  # For notebook testing, use test API keys
    api_secret=config["api_secret"],
    testnet=True,  # Use testnet for safe testing
)

# Test connection
account_info = exchange.get_account_info()
print(f"Connected to {config['exchange']} testnet")
print(f"Account Balance: {account_info['balance']} {account_info['currency']}")

## 3. Load Trader Instance for Monitoring

In [ ]:
# Initialize strategy
strategy = create_strategy(
    strategy_type=config["strategy"], config=config, model_path=f"../models/{config['strategy']}_model.pth"
)

# Initialize trader
trader = Trader(strategy=strategy, exchange=exchange, config=config)

print(f"Initialized {config['strategy']} strategy for live trading")

## 4. Monitor Trading Positions

In [ ]:
# Get current positions
positions = exchange.get_positions()

if positions:
    # Display positions
    positions_df = pd.DataFrame(positions)
    positions_df

In [ ]:
# Get recent market data
recent_data = exchange.get_historical_data(symbol=config["symbol"], timeframe=config["timeframe"], limit=100)

# Plot recent price action
plt.figure(figsize=(14, 7))
plt.plot(recent_data.index, recent_data["close"])
plt.title(f"{config['symbol']} Recent Price Action")
plt.xlabel("Time")
plt.ylabel("Price")
plt.grid(True)
plt.show()

## 5. Strategy Signal Analysis

In [ ]:
# Generate signals on recent data
signals = strategy.generate_signals(recent_data)

# Plot price with signals
plt.figure(figsize=(14, 7))
plt.plot(recent_data.index, recent_data["close"], label="Price")

# Plot buy signals
buy_signals = signals[signals > 0]
if not buy_signals.empty:
    plt.scatter(
        buy_signals.index,
        recent_data.loc[buy_signals.index, "close"],
        marker="^",
        color="green",
        s=100,
        label="Buy Signal",
    )

# Plot sell signals
sell_signals = signals[signals < 0]
if not sell_signals.empty:
    plt.scatter(
        sell_signals.index,
        recent_data.loc[sell_signals.index, "close"],
        marker="v",
        color="red",
        s=100,
        label="Sell Signal",
    )

plt.title(f"{config['strategy']} Strategy Signals")
plt.xlabel("Time")
plt.ylabel("Price")
plt.legend()
plt.grid(True)
plt.show()

## 6. Performance Tracking

In [ ]:
# Load trade history from exchange
trade_history = exchange.get_trade_history(
    symbol=config["symbol"],
    start_time=(datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d"),
    end_time=datetime.now().strftime("%Y-%m-%d"),
)

if trade_history:
    # Convert to DataFrame
    # Convert to DataFrame and ensure 'profit' column exists (assuming 'pnl' is profit)
    trades_df = pd.DataFrame(trade_history)
    if "pnl" in trades_df.columns and "profit" not in trades_df.columns:
        trades_df["profit"] = trades_df["pnl"]

    # Ensure timestamp is datetime for sorting
    trades_df["timestamp"] = pd.to_datetime(trades_df["timestamp"])
    trades_df = trades_df.sort_values("timestamp")

    # Construct equity curve (assuming daily aggregation for simplicity)
    # Note: This is an approximation. A true equity curve needs point-in-time portfolio values.
    initial_capital = config.get("initial_capital", 10000)  # Get initial capital from config or default
    daily_pnl = trades_df.set_index("timestamp")["profit"].resample("D").sum()
    equity_curve_values = [initial_capital] + (initial_capital + daily_pnl.cumsum()).tolist()

    # Convert trades DataFrame back to list of dicts for calculate_metrics
    trades_list = trades_df.to_dict("records")

    # Calculate metrics using the function from btb.backtest
    live_metrics = calculate_metrics(equity_curve=equity_curve_values, trades=trades_list, config=config)

    # Display calculated metrics
    print("--- Live Performance Metrics ---")
    print(f"Initial Capital: {live_metrics.get('initial_capital', 'N/A'):.2f}")
    print(f"Final Capital: {live_metrics.get('final_capital', 'N/A'):.2f}")
    print(f"Total Return: {live_metrics.get('total_return_pct', 'N/A'):.2f}%")
    print(f"Total Trades: {live_metrics.get('total_trades', 'N/A')}")
    print(f"Win Rate: {live_metrics.get('win_rate', 0) * 100:.2f}%")
    print(f"Profit Factor: {live_metrics.get('profit_factor', 'N/A'):.2f}")
    print(f"Max Drawdown: {live_metrics.get('max_drawdown_pct', 'N/A'):.2f}%")
    print(f"Sharpe Ratio: {live_metrics.get('sharpe_ratio', 'N/A'):.2f}")
    print(f"Sortino Ratio: {live_metrics.get('sortino_ratio', 'N/A'):.2f}")
    print(f"Avg Profit (%): {live_metrics.get('avg_profit_pct', 0) * 100:.2f}%")
    print(f"Avg Loss (%): {live_metrics.get('avg_loss_pct', 0) * 100:.2f}%")

    # Display recent trades DataFrame
    print("\n--- Recent Trades ---")
    print(trades_df.head())
else:
    print("No trade history available for the specified period.")

## 7. Risk Management Analysis

The `calculate_metrics` function already computes key risk metrics like Max Drawdown. Additional risk analysis (e.g., VaR, position exposure over time) could be added here if needed, potentially requiring more detailed position or equity data than typically available in simple trade history.

## 8. Compare Live vs Backtest Performance

In [ ]:
# Load backtest results (assuming previously saved)
# backtest_results = pd.read_csv('../data/backtest_results.csv', index_col=0, parse_dates=True)

# Uncomment below to compare live vs backtest performance (if data available)

# if trade_history and 'backtest_results' in locals():
#     # Prepare live trading data
#     live_start_date = trades_df['timestamp'].min().date()
#     live_end_date = trades_df['timestamp'].max().date()
#
#     # Filter backtest results to match live trading period
#     backtest_filtered = backtest_results[
#         (backtest_results.index.date >= live_start_date) &
#         (backtest_results.index.date <= live_end_date)
#     ]
#
#     # Convert live trades to equity curve
#     live_daily = trades_df.groupby('date')['pnl'].sum()
#     live_equity = config['initial_capital'] + live_daily.cumsum()
#
#     # Plot comparison
#     plt.figure(figsize=(14, 7))
#     plt.plot(backtest_filtered.index, backtest_filtered['equity'], label='Backtest')
#     plt.plot(live_equity.index, live_equity.values, label='Live Trading')
#     plt.title('Backtest vs Live Trading Performance')
#     plt.xlabel('Date')
#     plt.ylabel('Equity')
#     plt.legend()
#     plt.grid(True)
#     plt.show()

## 9. Market Condition Analysis

In [ ]:
# Get volatility over time
recent_data["daily_return"] = recent_data["close"].pct_change()
recent_data["volatility"] = recent_data["daily_return"].rolling(window=14).std() * np.sqrt(365)

# Plot volatility
plt.figure(figsize=(14, 10))

plt.subplot(2, 1, 1)
plt.plot(recent_data.index, recent_data["close"])
plt.title("Price")
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(recent_data.index, recent_data["volatility"])
plt.title("Annualized Volatility (14-day rolling)")
plt.grid(True)

plt.tight_layout()
plt.show()

## 10. Real-Time Monitoring

# 10. Real-Time Monitoring

Monitoring a live `Trader` instance typically involves checking logs, database records, or a dedicated monitoring API/dashboard rather than running a loop within a notebook. The cells below demonstrate analyzing past performance based on trade history.

## 11. Strategy Adjustments

In [ ]:
# Function to adjust strategy parameters based on market conditions
def adjust_strategy_parameters(volatility_threshold=0.5):
    """Adjust strategy parameters based on market conditions"""
    # Get recent data for analysis
    recent_data = exchange.get_historical_data(symbol=config["symbol"], timeframe=config["timeframe"], limit=100)

    # Calculate current volatility
    recent_data["daily_return"] = recent_data["close"].pct_change()
    current_volatility = recent_data["daily_return"].std() * np.sqrt(365)

    print(f"Current annualized volatility: {current_volatility:.2f}")

    # Adjust parameters based on volatility
    if current_volatility > volatility_threshold:
        # High volatility - reduce position size, increase signal threshold
        adjusted_config = config.copy()
        adjusted_config["position_size"] = config["position_size"] * 0.7  # Reduce position size by 30%
        adjusted_config["signal_threshold"] = config["signal_threshold"] * 1.5  # Increase threshold by 50%

        print("High volatility detected. Adjusting strategy parameters:")
        print(f"Position size: {config['position_size']} -> {adjusted_config['position_size']}")
        print(f"Signal threshold: {config['signal_threshold']} -> {adjusted_config['signal_threshold']}")

        return adjusted_config
    else:
        print("Normal volatility. Using default parameters.")
        return config

In [ ]:
# Uncomment to test strategy parameter adjustment
# adjusted_config = adjust_strategy_parameters(volatility_threshold=0.3)

## 12. Summary and Next Steps

In this notebook, we:

1. Connected to a trading exchange (testnet) and initialized our trading strategy
2. Monitored current positions and recent market data
3. Analyzed strategy signals on recent data
4. Tracked live trading performance and compared it with backtest expectations
5. Analyzed risk metrics including drawdowns and volatility
6. Implemented real-time monitoring functionality
7. Demonstrated adaptive strategy parameter adjustment based on market conditions

Next steps for live trading deployment:
- Set up a dedicated server for running the trading bot 24/7
- Implement proper logging and error handling for unattended operation
- Configure notifications for important events (new trades, stops hit, etc.)
- Develop a dashboard for real-time monitoring of multiple strategies
- Implement automatic strategy adjustments based on performance metrics
- Set up regular performance reports and analytics

Remember to always start with small position sizes when going live, and gradually increase as the strategy proves consistent in live trading conditions.